In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Load the CSV file, without skipping any rows initially
file_path = "/Users/javanmardi/Work/IGSB/Bone2Gene_Survey_Study/3_progressive_results/B2G Survey_9_29_2024.csv"
initial_df = pd.read_csv(file_path, sep=';', header=None)

In [ ]:
# Set the first row as the header of the dataframe
headers = initial_df.iloc[0]
questions = initial_df.iloc[1]
header_question_dict = dict(zip(headers, questions))

In [ ]:
# Now, read the CSV again, this time skipping the first three rows and setting the first row as header
data_df = pd.read_csv(file_path, sep=';', skiprows=[1, 2])

# Show the first few rows of the dataframe to verify
# print("\nData Preview:")
data_df.head()

In [ ]:
#the finished surveys

finished_df=data_df[data_df['Finished']==True]
finished_df.shape

In [ ]:
# Identify empty columns
empty_columns = finished_df.isna().all()

# Drop empty columns
non_empty_columns = finished_df.drop(columns=empty_columns[empty_columns].index)
non_empty_columns.to_csv('non_empty_columns.csv', index=False)
non_empty_columns.shape

In [ ]:
# answered desired questions
question_columns = ['Q2', 'Q4', 'Q10', 'Q11','Q16','Q20','Q21','Q22','Q23','Q24','Q31','Q34','Q35']  # List of all question column names

# Count missing values in the specified columns for each row
missing_counts = non_empty_columns[question_columns].isna().sum(axis=1)

# Filter rows where the count of missing values is less than or equal to 3
data = non_empty_columns[missing_counts <= 3]
data.shape

# Q13

In [ ]:
header_question_dict['Q13']

In [ ]:
# Calculate frequency of values in the column
Q13_value_counts = data['Q13'].value_counts().reset_index()
Q13_value_counts.columns = ['Value', 'Count']
Q13_value_counts

In [ ]:
# Mapping dictionary
country_mapping = {
    'US': 'United States',
    'United States of America': 'United States',
    'USA': 'United States',
    'usa': 'United States',
    'united states': 'United States',
    'germany': 'Germany',
    'Deutschland': 'Germany',
    'portugal': 'Portugal',
    'canada': 'Canada',
    'FR': 'France',
    'italy': 'Italy',
    'ITALY': 'Italy',
    'Moscow': 'Russia',
    'Slowenien': 'Slovenia',
    'Turkiye ':'Turkey',
    'TÃ¼rkiye':'Turkey',
    'BELGIUM':'Belgium',
    'Austria (Europe)':'Austria',
    'UK':'United Kingdom',
    'United Kingdom ':'United Kingdom'
}

# Replace the country names in the DataFrame
# data['Q13'] = data['Q13'].replace(country_mapping)

# Replace the country names in the DataFrame using .loc for safe and explicit indexing
data.loc[:, 'Q13'] = data['Q13'].replace(country_mapping)



In [ ]:
np.sum(Q13_value_counts['Count'])

# Figure

In [ ]:
# Create a world map using Plotly Express scatter_geo
fig = px.scatter_geo(Q13_value_counts, locations="Value", locationmode='country names', color="Count", size="Count",
                     projection="natural earth", color_continuous_scale="Jet_r", size_max=60)

# Center align the title
fig.update_layout(title={
    'text': 'Participants locations',
    'x': 0.5,  # Align center
    'xanchor': 'center',
    'yanchor': 'top',
    'font': {'size': 24}  # Increase font size
})

# Remove the margins
fig.update_layout(margin=dict(l=0, r=0, t=50, b=10),
    width=1600,  # Set the width of the figure
    height=600,  # Set the height of the figure
    coloraxis_colorbar={
        'title': 'Count',  # Define your color bar title here
        'title_font': {'size': 20},  # Increase the font size of the color bar title
        'tickfont': {'size': 16}  # Increase the font size of the tick values
    }
)


fig.show()

fig.write_image("world_map.png", width=1600, height=750, scale=6)

# Histogram

In [ ]:
fig = go.Figure(
    go.Bar(y=Q13_value_counts['Count'], x=Q13_value_counts['Value'], orientation='v', showlegend=False, 
           text=Q13_value_counts['Count'], textposition='auto', textfont=dict(size=24))

)

# bar_chart_1_order = ['Less than 5', '5 to 10', '10 to 20', '20 to 50', '50 to 100', '100 to 200', 'Greater than 200']
# fig.update_xaxes(categoryorder='array', categoryarray=bar_chart_1_order)


fig.update_xaxes(showgrid=True, zeroline=True, showline=True, linewidth=1, linecolor='black', mirror=False, tickfont=dict(size=24), tickangle=40)
fig.update_yaxes(showgrid=True, zeroline=True, showline=False, linewidth=1, linecolor='black', mirror=False, tickfont=dict(size=24), visible=False)

fig.update_layout(
    width=1600,   # Set the width of the figure
    height=400   # Set the height of the figure
)

fig.update_layout(
    paper_bgcolor="white",
    plot_bgcolor="white",
    title_text="",  # Add a title to the figure Number of participants per country
    title_x=0.5,  # Center the title horizontally
    title_font=dict(size=30),
    margin=dict(t=50, l=50, r=50, b=50)
)

fig.show()
fig.write_image("countries_histogram.png", width=1600, height=400, scale=6)

# Continents

In [ ]:
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2


In [ ]:
# Function to get continent from country name
def get_continent(country):
    try:
        alpha2 = country_name_to_country_alpha2(country)
        continent_code = country_alpha2_to_continent_code(alpha2)
        return continent_code
    except:
        return 'Unknown'  # for countries that are not found

# Apply the function to create a new continent column
Q13_value_counts['Continent'] = Q13_value_counts['Value'].apply(get_continent)

# Group by continent and sum the counts
result = Q13_value_counts.groupby('Continent')['Count'].sum().reset_index()

print(result)

In [ ]:
Q13_value_counts